### Логистическая регрессия


Всем привет, меня зовут Хворостяный Вячеслав. В этой статье я продолжу тему имплементации базовых алгоритмов "машинного обучения" с помощью Python. В предидущей статье был расмотрен алгоритм линейной регресии, а также такой метод оптимизации как градиентный спуск. В этой статье разберем логистическую регрессию, регуляризацию и применения алгоритмов на практике.

Существует два основных типа алгоритмов "машинного обучения" - "обучение с учетелем" ("supervised learning") и "обучение без учителя" ("unsupervised learning"). В свою очередь, "обучение с учителем" делиться на регрессионные алгоритмы и алгоритмы класификации. Логистическая регрессия пренадлежит к типу "supervised learning", и, несмотря на название, является алгоритмом класификации, часто используемым как функция активации в нейронных сетях, или как самостоятельный метод класификации объектов. "Обучение с учителем" значит, что у нас уже есть некоторое количество данных где известны как значение аргументов, так и значения самой функции, и наша задача состоит в том, чтобы "обучится" на этих, уже размеченных данных.

В основе алгоритма логистической регрессии лежит экспоненциальная кривая или сигмоида, которая разделяет объекты на два класса.

$$z^{(i)} = w^T x^{(i)} + b $$
 
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})$$

$$sigmoid(z^{(i)}) = \frac{1}{1-e^{-z^{(i)}}}$$



Где   
$z^{(i)}$ - функция прямой (линейная регрессия)  
$\hat{y}^{(i)}$ - гипотеза   
$sigmoid(z^{(i)})$ - сигмоида 

Задача алгоритма - вичислить вероятность того, что елемент пренадлежит к одному из классов. Значения функции находятся в границах от [0,1], самый простой способ - округлить все значения больше 0.5 до единицы,порог можно задать и выше, в том случае, если верно определить позитивный класс является задачей более критичной чем отличить красный носок от белого.

Кост-функция в случае логистической регрессии будет выглядеть так:
$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m - y^{(i)} \log(a^{(i)}) -  (1-y^{(i)} ) \log(1-a^{(i)})$$



Датасет может иметь много полей или "фичей", но не все фичи вносят одинаковый вклад в принятие алгоритмом решения. По сути признаки (фичи) с малим вкладом являются "шумом", который накладывает негативный след на работу алгоритма, не позволяя модели обобщить обсчеты. Этот эффект называют "переобучение" или "overfitting", алгоритм "заучивается" на текущих данных и плохо работает с новимы. В свою очередь недостаточное количество фичей вызывает "недообучение" или "underfitting", модель плохо показывает себя как на тренинг сете, так и на тесте, сложно оценить стоимость автомобиля, зная только что у него сиденья с подогревом, согласитесь. Серебряной пули нет, и к каждой проблеме нужно подходить индивидуально, но при большом количестве фичей регуляризация спасает от обучения модели на "шуме".

### Регуляризация
Регуляризация - это подход, который позволяет снизить сложность модели за счет "штрафования" вектора параметров $\theta$, один из эфективных методов борьбы с "переобучением", наряду с кросс-валидацией и уменьшением количества фичей, о которых мы поговорим позже. Регуляризация дает возможность выделить фичи,которые вносят наибольший вклад в принятия решения, и снизить влияние фич создающих "шум". Существует два вида регуляризации - L1 и L2, выбор вида регуляризации отвечает на вопрос "как штрафовать". Рассмотрим различия между ними.

##### L1 и L2 регуляризация
  
  
**L1 регуляризация** 

L1 регуляризация или Лассо предсталена в виде суммы модулей всех елементов вектора праметров $\theta$, L1 нормы. Этот тип регуляризации хорошо показивает себя на простых моделях, она устойчивс к выбросам (outliers), "недорогая" с точки зрения вычислительных операций, хороша для прореживания фичей, сводя к нулю менее существенные. L1 регуляризация часто используется для выбора фичей ("feature selection").

$$J(\theta) = \frac{1}{m} \sum_{i=1}^m - y^{(i)} \log(a^{(i)}) -  (1-y^{(i)} ) \log(1-a^{(i)}) + \boxed{\lambda  \sum_{i=1}^m \mid\theta^{(i)}\mid} $$
Где $\lambda$ - гиперпараметр, определяющий насколько "штрафовать" вектор параметров, подбирается в ручную. 

**L2 регуляризация**   
L2 регуляризация является суммой квадратов всех елементов вектора параметров $\theta$.
Применяется к сложным моделям, не устойчива к выбросам, не сводит значения вектора параметров к нулю, и не прореживает фичи, но в отличии от L1 хорошо показывает себя когда все входные фичи имеют размерность одного порядка.

$$J(\theta) = \frac{1}{m} \sum_{i=1}^m - y^{(i)} \log(a^{(i)}) -  (1-y^{(i)} ) \log(1-a^{(i)}) + \boxed{\lambda  \sum_{i=1}^m \theta^{(i)^2}} $$

Имплементация на Python

```python
def sigmoid(z):
    return 1/ (1 + np.exp(-z))

def costFunctionReg(theta, X, y ,Lambda):
    m=len(y)
    y=y[:,np.newaxis]
    predictions = sigmoid(X @ theta)
    error = (-y * np.log(predictions)) - ((1-y)*np.log(1-predictions))
    cost = 1/m * sum(error)
    regCost= cost + Lambda/(2*m) * sum(theta**2)
    
    j_0= 1/m * (X.transpose() @ (predictions - y))[0]
    j_1 = 1/m * (X.transpose() @ (predictions - y))[1:] + (Lambda/m)* theta[1:]
    grad= np.vstack((j_0[:,np.newaxis],j_1))
    return regCost[0], grad


def gradientDescent(X,y,theta,alpha,num_iters,Lambda):
    m=len(y)
    J_history =[]
    
    for i in range(num_iters):
        cost, grad = costFunctionReg(theta,X,y,Lambda)
        theta = theta - (alpha * grad)
        J_history.append(cost)    
    return theta , J_history
theta , J_history = gradientDescent(X,y,initial_theta,1,800,0.2)
print("The regularized theta using ridge regression:\n",theta)

```